##### Copyright 2020 Google Inc.

Licensed under the Apache License, Version 2.0 (the "License").
<!--
    Licensed to the Apache Software Foundation (ASF) under one
    or more contributor license agreements.  See the NOTICE file
    distributed with this work for additional information
    regarding copyright ownership.  The ASF licenses this file
    to you under the Apache License, Version 2.0 (the
    "License"); you may not use this file except in compliance
    with the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing,
    software distributed under the License is distributed on an
    "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    KIND, either express or implied.  See the License for the
    specific language governing permissions and limitations
    under the License.
-->


# Example 1: Word Count

This example demonstrates how to set up an Apache Beam pipeline that reads from a
[Google Cloud Storage](https://cloud.google.com/storage) file containing text from Shakespeare's work *King Lear*, 
tokenizes the text lines into individual words, and performs a frequency count on each of those words. 

An Apache Beam pipeline is a pipeline that reads input data, transforms that
data, and writes output data. It consists of `PTransform`s and `PCollection`s.
A `PCollection` represents a distributed data set that your Beam pipeline operates on.
A `PTransform` represents a data processing operation, or a step, in your pipeline.
It takes one or more `PCollection`s as input, performs a processing function
that you provide on the elements of that `PCollection`, and produces zero
or more output `PCollection` objects.

For details about Apache Beam pipelines, including `PTransform`s and
`PCollection`s, visit [Beam Programming Guide](https://beam.apache.org/documentation/programming-guide/).

You'll be able to use this notebook to explore the data in each `PCollection`.

We first start with the necessary imports:

In [70]:
# Python's regular expression library
import re

# Beam and interactive Beam imports
import apache_beam.io.fileio
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.runners.dataflow.dataflow_runner import DataflowRunner
import apache_beam.runners.interactive.interactive_beam as ib

The following defines a `PTransform` named `ReadWordsFromText`, that extracts words from a file.

In [23]:
class ReadWordsFromText(beam.PTransform):
    
    def __init__(self, file_pattern):
        self._file_pattern = file_pattern
    
    def expand(self, pcoll):
        return (pcoll.pipeline
                | beam.io.ReadFromText(self._file_pattern)
                | beam.FlatMap(lambda line: re.findall(r'[\w\']+', line.strip(), re.UNICODE)))

The following sets up an Apache Beam pipeline with the *Interactive Runner*.
The *Interactive Runner* is the runner suitable for running in notebooks.
A runner is an execution engine for Apache Beam pipelines.

In [33]:
p = beam.Pipeline(InteractiveRunner())

The following sets up a `PTransform` that extracts words from a Google Cloud Storage file that contains the text of Shakespeare's work *King Lear*.

`|` is an overloaded operator that applies a `PTransform` to a `PCollection` to produce a new `PCollection`.
Together with `|`, `>>` allows you to optionally name a `PTransform`.

Usage: `<PCollection> | <PTransform>` or `<PCollection> | <name> >> <PTransform>`


In [28]:
from apache_beam.options.pipeline_options import PipelineOptions

options = PipelineOptions(
    runner='DataflowRunner',
    project='curatedmetagenomicdata',
    job_name='genefamilies_relab',
    temp_location='gs://cmgd-data/temp',
    region='us-central1')

f = (p 
     | beam.io.fileio.MatchFiles('gs://cmgd-data/manual/AsnicarF_2017/marker_/MV_FEI1_t1Q14.tsv') 
     | beam.io.WriteToText('gs://cmgd-data/df/abc'))


The following sets up a `PTransform` to count the words. `counts` is a `PCollection` that will contain the count data.


In [104]:
import csv
import apache_beam as beam
import json

# Python's regular expression library
import re

# Beam and interactive Beam imports
import apache_beam.io.fileio
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.runners.dataflow.dataflow_runner import DataflowRunner
import apache_beam.runners.interactive.interactive_beam as ib

import logging

from apache_beam.options.pipeline_options import PipelineOptions

options = PipelineOptions(
    runner='DataflowRunner',
    project='curatedmetagenomicdata',
    job_name='marker-presence', #no underlines allowed
    temp_location='gs://cmgd-data/temp',
    staging_location='gs://cmgd-data/temp',
    region='us-central1')

class CSVFixer(beam.DoFn):
    def process(self, fobj):
        import apache_beam as beam
        res = []
        parts = fobj.path.split('/')
        study_id=parts[4]
        filetype = parts[5]
        sample_id=parts[6].replace('.tsv','')
        with beam.io.gcp.gcsfilesystem.GCSFileSystem({}).open(fobj.path,'r') as f:
            logging.info(study_id,filetype,sample_id)
            for r in f:
                row = r.decode('UTF-8')
                if(row.startswith('#')):
                    continue
                rowsplit=row.strip().split('\t')
                
                # marker_abundance
                #d = {
                #    'marker': rowsplit[0],
                #    'abundance': float(rowsplit[1])
                #}  
                
                # marker_presence
                d = {
                    'marker': rowsplit[0],
                    'presence': float(rowsplit[1])>0 # there is an odd 7.007007007 that shows up here
                }  
                
                # pathabundance_relab
                #namesplit = rowsplit[0].split('|')
                #d = {
                #    'pathway': namesplit[0],
                #    'unclassified': len(namesplit)>1,
                #    'relative_abundance': float(rowsplit[1])
                #}
                
                #pathcoverage 
                #namesplit = rowsplit[0].split('|')
                #d = {
                #    'pathway': namesplit[0],
                #    'unclassified': len(namesplit)>1,
                #    'covered': bool(int(rowsplit[1]))
                #}
                
                # genfamilies_relab
                #d = {
                #    'taxon': rowsplit[0].split(b'|')[-1],
                #    'relative_abundance': float(rowsplit[1]) #,
                #    #'ncbi_taxonomy': rowsplit[1].split(b'|')[-1]
                #}
                d.update({
                    'study_id':study_id,
                    'sample_id':sample_id,
                    'filetype': filetype
                })
                res.append(d)
        return res
    


#p = beam.Pipeline(InteractiveRunner())

gcs = beam.io.gcp.gcsfilesystem.GCSFileSystem({})
#var = 'genefamilies_relab'
#var = 'pathabundance_relab'
#var = 'pathcoverage'
#var = 'marker_abundance'
ver = 'marker_presence'
with beam.Pipeline(options=options) as p:
    (p | beam.io.fileio.MatchFiles(f'gs://cmgd-data/manual/**/{var}/*')
     | beam.Reshuffle()
     | beam.ParDo(CSVFixer())
     | beam.Map(lambda x: json.dumps(x))
     | beam.io.WriteToText(f'gs://cmgd-data/temp/{var}.json')
    )
#p.run()

In [103]:
import csv
import apache_beam as beam
import json

# Python's regular expression library
import re

# Beam and interactive Beam imports
import apache_beam.io.fileio
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.runners.dataflow.dataflow_runner import DataflowRunner
import apache_beam.runners.interactive.interactive_beam as ib

import logging

from apache_beam.options.pipeline_options import PipelineOptions

options = PipelineOptions(
    runner='DataflowRunner',
    project='curatedmetagenomicdata',
    job_name='metaphlan-bugs-list',
    temp_location='gs://cmgd-data/temp',
    staging_location='gs://cmgd-data/temp',
    region='us-central1')

class CSVFixer(beam.DoFn):
    def process(self, fobj):
        import apache_beam as beam
        res = []
        parts = fobj.path.split('/')
        study_id=parts[4]
        filetype = parts[5]
        sample_id=parts[6].replace('.tsv','')
        with beam.io.gcp.gcsfilesystem.GCSFileSystem({}).open(fobj.path,'r') as f:
            logging.info(study_id,filetype,sample_id)
            for r in f:
                row = r.decode('UTF-8')
                if(row.startswith('#')):
                    continue
                rowsplit=row.strip().split('\t')
                abund = None
                try:
                    abund=float(rowsplit[2])
                except:
                    pass
                ncbi_taxon_id=None
                try:
                    ncbi_taxon_id=int(rowsplit[1].split('|')[-1])
                except:
                    pass
                d = {
                    'clade_string': rowsplit[0],
                    'taxonomy_string': rowsplit[1],
                    'clade': rowsplit[0].split('|')[-1],
                    'relative_abundance': abund,
                    'ncbi_taxonomy': ncbi_taxon_id
                }
                d.update({
                    'study_id':study_id,
                    'sample_id':sample_id,
                    'filetype': filetype
                })
                res.append(d)
        return res

#p = beam.Pipeline(InteractiveRunner())

gcs = beam.io.gcp.gcsfilesystem.GCSFileSystem({})
with beam.Pipeline(options=options) as p:
#with beam.Pipeline(InteractiveRunner()) as p:
    (p | beam.io.fileio.MatchFiles('gs://cmgd-data/manual/**/metaphlan_bugs_list/*')
     | beam.Reshuffle()
     | beam.ParDo(CSVFixer())
     | beam.Map(lambda x: json.dumps(x))
     | beam.io.WriteToText('gs://cmgd-data/temp/metaphlan_bugs_list.json')
    )
#p.run()

KeyboardInterrupt: 

The following implicitly runs the pipeline and shows the elements in `PCollection` `count`.

In [56]:
'abc.txt'.replace('.txt','')

'abc'

The following sets up `PTransform`s that will convert the words to lowercase and then count them.

In [7]:
lower_counts = (words
                | "lower" >> beam.Map(lambda word: word.lower())
                | "lower_count" >> beam.combiners.Count.PerElement())

The following will return the count using the same words as before but with lowercase.
Because all words are converted to lowercase before being counted, some words
will have a higher count than before. 
(e.g. `KING: 2, King: 4, king: 3` will become `king: 9`)
Note the parameter `visualize_data=True`. This optional parameter gives you a visualization of the data (see [FAQ #3.How do I read the visualization](../../faq.md#q3)). 

In [14]:
(p | beam.io.ReadFromText('gs://cmgd-data/manual/AsnicarF_2017/genefamilies_relab/*.tsv') | beam.io.WriteToText('file:///tmp/abc.txt')) 

<PCollection[[14]: WriteToText/Write/WriteImpl/FinalizeWrite.None] at 0x7ff8606f7cd0>

The following gives you a [Pandas Dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) that represents the `PCollection` `lower_counts`.

In [12]:
f | beam.io.WriteToText('/tmp/abc') 

<PCollection[[12]: WriteToText/Write/WriteImpl/FinalizeWrite.None] at 0x7ff860812390>

You can see the job graph for the pipeline by doing:

In [ ]:
ib.show_graph(p)

This example is designed to run easily on a single machine. If you have many such files, add an output sink to your `PCollection` result by doing:
```
lower_counts | beam.io.WriteToText(<file>)
```
And if you have millions of such files with billions of words, you need a cluster of machines that have enough processing power and memory to finish processing them in a reasonable amount of time.
[Google Cloud Dataflow](https://cloud.google.com/dataflow) takes away the headache of managing such a cluster, parallelizes and reliably runs your Apache Beam pipelines, with intelligent auto-scaling so that you only pay for the resources needed for your pipelines.

Refer to [this walkthrough](Dataflow_Word_Count.ipynb) on how to run a Dataflow job using the example code in this notebook.

If you have any feedback on this notebook, drop us a line at beam-notebooks-feedback@google.com.